# Yuvraj Sharma: YAS001
# Numerical Methods
## Term Paper project: Lattice Boltzmann Method to Understand fluid flows

In [1]:
%matplotlib inline

In [2]:
from numpy import *
import matplotlib.pyplot as plt
from matplotlib import cm

# LBM Solver

## Initializing variables

In [3]:
maxIter = 170000                  # Total number of time iterations.
Re = 90                           # Reynolds number.
nx, ny = 420, 180                 # Numer of lattice nodes.
ly = ny-1                         # Height of the domain in lattice units.
cx, cy, L = nx//4, ny//2, ny//9   # Coordinates and characteristic length of the obstacle.
u_LB = 0.04                       # Velocity in lattice units.
n_LB = u_LB * L / Re;             # Viscosity in lattice units.
cs2 = 1/3                         # Speed of sound (lattice constant)
w = 1 / (n_LB / cs2 + 0.5);       # Relaxation parameter.

## Lattice Definitions

In [38]:
v = array([ [ 1,  1], [ 1,  0], [ 1, -1], [ 0,  1], [ 0,  0],
            [ 0, -1], [-1,  1], [-1,  0], [-1, -1] ])
t = array([ 1/36, 1/9, 1/36, 1/9, 4/9, 1/9, 1/36, 1/9, 1/36])

col1 = array([0, 1, 2])
col2 = array([3, 4, 5])
col3 = array([6, 7, 8])

## Function Definitions

In [39]:
def macroscopic(fin):
    """
    Returns rho and u for entire matrix (lattice)
    """
    rho = sum(fin, axis=0)
    u = zeros((2, nx, ny))
    for i in range(9):
        u[0,:,:] += v[i,0] * fin[i,:,:]
        u[1,:,:] += v[i,1] * fin[i,:,:]
    u /= rho
    return rho, u

def equilibrium(rho, u):
    """
    equilibrium distribution function E(i, rho, u) as in report
    """
    usqr = 1/(2*cs2) * (u[0]**2 + u[1]**2)
    E = zeros((9,nx,ny))
    for i in range(9):
        vu = 1/cs2 * (v[i,0] * u[0,:,:] + v[i,1] *u [1,:,:])
        E[i,:,:] = rho*t[i] * (1 + vu + 0.5 * vu**2 - usqr)
    return E

## Obstacle definitions

In [40]:
# Creation of a mask with T/F values, defining the shape of the obstacle.

def cylinder(x, y):
    return (x - cx)**2 + (y - cy)**2 < L**2

def square_pillar(x, y):
    return abs((x - cx) + (y - cy)) + abs((x - cx) - (y - cy)) < 2*L

def sheet(x, y):
    r = 20
    return abs(r*(x - cx) + (y - cy)) + abs(r*(x - cx) - (y - cy)) < 2*L

# Choosing the obstacle
obstacle = fromfunction(cylinder, (nx, ny))

## Initial velocity profile

In [41]:
# Almost zero, with a slight perturbation to trigger the instability.

def inivel(d, x, y):
    return (1-d) * u_LB * (1 + 1e-4*sin(y/ly*2*pi))

vel = fromfunction(inivel, (2,nx,ny))

# Initialization of the populations at equilibrium with the given velocity.
fin = equilibrium(1, vel)

## Main solver

In [43]:
# Main time loop
for time in range(maxIter):
    # Right wall: outflow condition.
    fin[col3,-1,:] = fin[col3,-2,:] 

    # Compute macroscopic variables, density and velocity.
    rho, u = macroscopic(fin)

    # Left wall: inflow condition.
    u[:,0,:] = vel[:,0,:]
    rho[0,:] = 1/(1-u[0,0,:]) * ( sum(fin[col2,0,:], axis=0) +
                                  2*sum(fin[col3,0,:], axis=0) )
    # Compute equilibrium.
    E = equilibrium(rho, u)
    fin[[0,1,2],0,:] = E[[0,1,2],0,:] + fin[[8,7,6],0,:] - E[[8,7,6],0,:]

    # Collision step.
    fout = fin - w * (fin - E)

    # Bounce-back condition for obstacle.
    for i in range(9):
        fout[i, obstacle] = fin[8-i, obstacle]

    # Streaming step.
    for i in range(9):
        fin[i,:,:] = roll(
                            roll(fout[i,:,:], v[i,0], axis=0),
                            v[i,1], axis=1 )
 
    # Visualization of the velocity.
    if (time%100==0):
        plt.clf()
        plt.imshow(sqrt(u[0]**2+u[1]**2).transpose(), cmap=cm.Purples)
        plt.savefig("/vel.{0:04d}.png".format(time//100))

# Making Videos

In [13]:
import cv2
import os

def images_to_video(image_folder, video_name, fps=24):
    images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
    images = np.sort(images)
    print(images)
    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = frame.shape

    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    for image in images:
        video.write(cv2.imread(os.path.join(image_folder, image)))

    cv2.destroyAllWindows()
    video.release()

image_folder = '/home/yuvi/icts/courses/Numerical Methods/term paper/NM Term Paper/lbm_wall_re_1100'
video_name = 'LBM_Re_1100.mp4'
fps = 24
images_to_video(image_folder, video_name, fps)


['vel.0000.png' 'vel.0001.png' 'vel.0002.png' ... 'vel.1504.png'
 'vel.1505.png' 'vel.1506.png']


In [14]:
image_folder = '/home/yuvi/icts/courses/Numerical Methods/term paper/NM Term Paper/lbm_circle_20'
video_name = 'LBM_Circle_Re_20.mp4'
fps = 24
images_to_video(image_folder, video_name, fps)

['vel.0000.png' 'vel.0001.png' 'vel.0002.png' ... 'vel.1397.png'
 'vel.1398.png' 'vel.1399.png']


In [20]:
image_folder = '/home/yuvi/icts/courses/Numerical Methods/term paper/NM Term Paper/lbm_circle_90'
video_name = 'LBM_Circle_Re_90.mp4'
fps = 24
images_to_video(image_folder, video_name, fps)

['vel.0000.png' 'vel.0001.png' 'vel.0002.png' ... 'vel.1497.png'
 'vel.1498.png' 'vel.1499.png']


In [19]:
image_folder = '/home/yuvi/icts/courses/Numerical Methods/term paper/NM Term Paper/lbm_triag_90'
video_name = 'LBM_Traig_Re_90.mp4'
fps = 30
images_to_video(image_folder, video_name, fps)

['vel.0000.png' 'vel.0001.png' 'vel.0002.png' ... 'vel.1497.png'
 'vel.1498.png' 'vel.1499.png']


In [18]:
image_folder = '/home/yuvi/icts/courses/Numerical Methods/term paper/NM Term Paper/lbm_triag_1100'
video_name = 'LBM_Traig_Re_1100.mp4'
fps = 24
images_to_video(image_folder, video_name, fps)

['vel.0000.png' 'vel.0001.png' 'vel.0002.png' ... 'vel.1497.png'
 'vel.1498.png' 'vel.1499.png']
